In [1]:
import requests
import pandas as pd

In [59]:
import boto3

In [2]:
link2 = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [3]:
link2

'https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad'

In [4]:
requisicao = requests.get(link2)

In [5]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 284.64,
  'feels_like': 284.19,
  'temp_min': 284.45,
  'temp_max': 286.34,
  'pressure': 1023,
  'humidity': 90,
  'sea_level': 1023,
  'grnd_level': 1018},
 'visibility': 10000,
 'wind': {'speed': 1.17, 'deg': 31, 'gust': 1},
 'clouds': {'all': 6},
 'dt': 1697262533,
 'sys': {'type': 2,
  'id': 2001569,
  'country': 'BR',
  'sunrise': 1697273279,
  'sunset': 1697319171},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [6]:
df = requisicao.json()

In [7]:
df1 = pd.json_normalize(df)

In [8]:
for i in df1:
    print(i)

weather
base
visibility
dt
timezone
id
name
cod
coord.lon
coord.lat
main.temp
main.feels_like
main.temp_min
main.temp_max
main.pressure
main.humidity
main.sea_level
main.grnd_level
wind.speed
wind.deg
wind.gust
clouds.all
sys.type
sys.id
sys.country
sys.sunrise
sys.sunset


In [9]:
df_lon_lat = pd.DataFrame(df['coord'], index=[0])

In [10]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [11]:
df_weather = pd.DataFrame(df['weather'], index=[0])

In [37]:
df_weather

,id,main,description,icon
0,800,Clear,clear sky,01n


In [13]:
df_main = pd.DataFrame(df['main'], index=[0])

In [14]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,284.64,284.19,284.45,286.34,1023,90,1023,1018


In [23]:
df1 = pd.DataFrame(df1)

In [25]:
df1

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1697262533,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1018,1.17,31,1,6,2,2001569,BR,1697273279,1697319171


In [32]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [35]:
df_clumns_selected = df1[selected_columns]

In [36]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1697262533,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,284.64,...,1018,1.17,31,1,6,2,2001569,BR,1697273279,1697319171


In [53]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [66]:
super_df.to_csv("C:/Users/crist/Desktop/s3_testes/super_df.csv")

c:\Envs\data-analysis\Lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


putaria com o boto3

In [73]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')


In [61]:
s3

In [74]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [75]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.csv"

In [76]:
s3_file_name = 'Catuaba010101'

In [72]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)